In [1]:

from components.controller.bluetooth_controller import BlueToothCarControlSPP
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2
from components.gyroscope.gyroscope import AngularSpeedControlV2
from components.two_wheels import TwoWheelsV2
from components.camera import PicameraV2
from components.controller.image_ml import ImageMLController
from data_collection.data_collection import LoggerSet, Logger
from multiprocessing import Manager

from components import default_loop_v2, default_component_process_starter_v2, shared_value

import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)
from ctypes import c_double



%load_ext autoreload
%autoreload 2

In [3]:
logger_set = LoggerSet('../log/session0', overwrite_ok=False)
manager = Manager()

## Two Wheels

In [6]:
w_out, w_starter = default_component_process_starter_v2(
    TwoWheelsV2, 
    init_kwargs= dict(
        left_pin=33, 
        right_pin=32, 
        dir_pins=(16, 18), 
        logger_set=logger_set, 
        name='TwoWheelsV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
)
imu_out, imu_starter = default_component_process_starter_v2(
    AngularSpeedControlV2, 
    init_kwargs= dict(
        logger_set=logger_set, 
        name='AngularSpeedControlV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
)
cam_out, cam_starter = default_component_process_starter_v2(
    PicameraV2, 
    init_kwargs= dict(
        resolution=(114, 64), framerate=30,
        logger_set=logger_set, 
        name='PicameraV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
    shared_outputs_kwargs=dict(resolution=(114, 64))
)

bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)

bt_out, bt_starter = default_component_process_starter_v2(
    BlueToothCarControlSPP, 
    init_kwargs= dict(
        logger_set=logger_set, 
        name='BlueToothCarControlSPP'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/30), 
    manager=manager, 
)


w = w_starter(imu_out)
imu = imu_starter(bt_out)
cam = cam_starter()
bt = bt_starter(bt_ser_out)

Starting Serial Port Profile...


/home/kawa/projects/car1/src/components/two_wheels.py:15: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:73: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:74: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
Process Process-9:


Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/kawa/projects/car1/src/components/__init__.py", line 30, in default_loop_v2
    obj = instantiater(**init_kwargs, **other_io)
Process Process-8:
  File "/home/kawa/projects/car1/src/components/gyroscope/gyroscope.py", line 241, in entry
    logger = logger_set.get_logger(name, **kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/kawa/projects/car1/src/data_collection/data_collection.py", line 27, in get_logger
    return Logger(self.path, name, save_interval=save_interval, overwrite_ok=self.overwrite_ok)
Process Process-10:
  File "/usr/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **s

NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 105)


In [5]:
w.terminate()
imu.terminate()
cam.terminate()
bt_ser.terminate()
bt.terminate()


# ML Component


In [186]:
ml_out, ml_starter = default_component_process_starter_v2(
    ImageMLController, 
    init_kwargs= dict(
        #resolution=(114, 64), framerate=30,
        model_path='test.keras',
        logger_set=logger_set, 
        name='ImageMLController'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
)
ml = ml_starter(cam_out)

In [194]:
ml.terminate()